In [1]:
import random
import tqdm

In [26]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    for x in x_split:
        features[x] = features.get(x, 0) + 1.0
    return features

In [27]:
feature_weights = {}

In [28]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [29]:
x_train, y_train = read_xy_data('Data/sst-sentiment-text-fiveclass/test.txt')
x_dev, y_dev = read_xy_data('Data/sst-sentiment-text-fiveclass/dev.txt')

In [30]:
print(x_train[0])
print(y_train[0])

Effective but too-tepid biopic
2


In [31]:
def run_classifier(features: dict[str, float]) -> int:
    score = 0
    for feat_name, feat_value in features.items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

In [32]:
NUM_EPOCHS = 200
for epoch in range(1, NUM_EPOCHS+1):
    # Shuffle the order of the data
    data_ids = list(range(len(x_train)))
    random.shuffle(data_ids)
    # Run over all data points
    for data_id in tqdm.tqdm(data_ids, desc=f'Epoch {epoch}'):
        x = x_train[data_id]
        y = y_train[data_id]
        # We will skip neutral examples
        if y == 0:    
            continue
        # Make a prediction
        features = extract_features(x)
        predicted_y = run_classifier(features)
        # Update the weights if the prediction is wrong
        if predicted_y != y:
            for feature in features:
                feature_weights[feature] = feature_weights.get(feature, 0) + y * features[feature]

Epoch 68: 100%|█████████████████████████████████████████████████████████████████| 2210/2210 [00:00<00:00, 75626.07it/s]


Epoch 136: 100%|████████████████████████████████████████████████████████████████| 2210/2210 [00:00<00:00, 54702.29it/s]


Epoch 200: 100%|████████████████████████████████████████████████████████████████| 2210/2210 [00:00<00:00, 45866.15it/s]


In [33]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(extract_features(x))
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [34]:
label_count = {}
for y in y_dev:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)

{3: 279, 2: 229, 4: 165, 0: 139, 1: 289}


In [35]:
train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_dev, y_dev)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.2864253393665158
Dev/test accuracy: 0.26248864668483196
